In [1]:
import requests
import json
from math import ceil
import time
import psycopg2 as pg
import pandas as pd
import numpy as np
import io

In [2]:
API_KEY = 'S47aT37T3flUeMeR1-UisjDOYz1atYlD69BDiqvOhXTZb1dYdvDemGueu5vZxh5gvNWCx1OpOfXPLWmdAb-LiiI_EhU0nC8g8ES0sDGgV9H1FDhEl0PiDR_g5zPYZXYx'


In [4]:
engine = pg.connect("dbname='airflow' user='postgres' host='localhost' port='5432' password='admin'")
df = pd.read_sql('select * from common.zips', con=engine)

C:\Users\86136\AppData\Local\Temp\ipykernel_35312\1044340143.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from common.zips', con=engine)


In [5]:
df_la = df[df.county == 'Los Angeles County']
df_la

,zip,city,state,county,country
38211,90001,Los Angeles,CA,Los Angeles County,US
38212,90002,Los Angeles,CA,Los Angeles County,US
38213,90003,Los Angeles,CA,Los Angeles County,US
38214,90004,Los Angeles,CA,Los Angeles County,US
38215,90005,Los Angeles,CA,Los Angeles County,US
...,...,...,...,...,...
39567,93584,Lancaster,CA,Los Angeles County,US
39568,93586,Lancaster,CA,Los Angeles County,US
39569,93590,Palmdale,CA,Los Angeles County,US
39570,93591,Palmdale,CA,Los Angeles County,US


In [8]:
df_la.city.unique().tolist()

['Los Angeles',
 'West Hollywood',
 'Dodgertown',
 'Playa Vista',
 'Bell Gardens',
 'Bell',
 'Beverly Hills',
 'Compton',
 'Culver City',
 'Downey',
 'El Segundo',
 'Gardena',
 'Hawthorne',
 'Hermosa Beach',
 'Huntington Park',
 'Lawndale',
 'Lynwood',
 'Malibu',
 'Manhattan Beach',
 'Maywood',
 'Pacific Palisades',
 'Palos Verdes Peninsula',
 'Rancho Palos Verdes',
 'Redondo Beach',
 'South Gate',
 'Topanga',
 'Venice',
 'Marina Del Rey',
 'Playa Del Rey',
 'Inglewood',
 'Santa Monica',
 'Torrance',
 'Whittier',
 'La Mirada',
 'Montebello',
 'Norwalk',
 'Pico Rivera',
 'Santa Fe Springs',
 'Artesia',
 'Cerritos',
 'Avalon',
 'Bellflower',
 'Harbor City',
 'Lakewood',
 'Hawaiian Gardens',
 'Lomita',
 'Paramount',
 'San Pedro',
 'Wilmington',
 'Carson',
 'Signal Hill',
 'Long Beach',
 'Altadena',
 'Arcadia',
 'Duarte',
 'La Canada Flintridge',
 'Monrovia',
 'Montrose',
 'Mount Wilson',
 'Sierra Madre',
 'South Pasadena',
 'Sunland',
 'Tujunga',
 'Verdugo City',
 'Pasadena',
 'San Marino

In [12]:
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [56]:
def api_call(city, offset):
    city_string = f"{'%20'.join(city.split(' '))}%2C%20LA"
    
    PARAM = {'offset': offset}
    url = f"https://api.yelp.com/v3/businesses/search?location={city_string}&sort_by=review_count&limit=50"
    headers = {
        "accept": "application/json",
        "Authorization": 'bearer %s' % API_KEY}

    response = requests.get(url = url, headers=headers, params=PARAM)
    return response

In [20]:
api_call_response = api_call('Maywood', offset = 0)

Maywood


In [23]:
api_call_response

<Response [200]>

In [49]:
def check_result(api_call_response, offset, city):
    retry = 0
    while api_call_response.status_code != 200:
        if retry >= 5:
            print('retries failed with ', city, offset)
            print('-------------')
            return {'status': -1, 'response': -1}
        
        else:
            print(f'error with city: {city} and offset: {offset}')
            time.sleep(10)
            api_call_result = api_call(city=city, offset=offset)
            retry+=1

    if api_call_response.status_code == 200:
        return {'status': 0, 'response': api_call_response}


In [13]:
city_list = df_la.city.unique().tolist()
city_list_chunk = list(divide_chunks(city_list, 3))

In [57]:
for chunk in city_list_chunk[:1]:
    all_res=[]
    error_res = []

    for city in chunk:
        print(city)
        api_call_response = api_call(city=city, offset=0)
        result = check_result(api_call_response=api_call_response, city=city, offset=0)
        if result['status'] == 0:
            total_res = json.loads(result['response'].text)['total']
            all_res = [json.loads(result['response'].text)['businesses']]
            print('total number of places', total_res)
        
            iter_num = ceil(min(1000, total_res)/50 - 1)
            print('total iterations',iter_num)
        else:
            error_res.append({'city': city, 
                         'offset': 0,
                         'result': result['response']})
            continue
        
        for i in range(1, iter_num):
            offset=i*50
            api_call_response = api_call(city=city, offset=offset)
            result = check_result(api_call_response=api_call_response, city=city, offset=offset)
            if result['status'] == 0:
                all_res += json.loads(result['response'].text)['businesses']
            else:
                error_res.append({'city': city, 
                         'offset': offset,
                         'result': result['response']})
            time.sleep(15)

Los Angeles
total number of places 15000
total iterations 19
error with city: Los Angeles and offset: 850
error with city: Los Angeles and offset: 850
error with city: Los Angeles and offset: 850
error with city: Los Angeles and offset: 850
error with city: Los Angeles and offset: 850
retries failed with  Los Angeles 850
-------------
error with city: Los Angeles and offset: 900
error with city: Los Angeles and offset: 900
error with city: Los Angeles and offset: 900
error with city: Los Angeles and offset: 900
error with city: Los Angeles and offset: 900
retries failed with  Los Angeles 900
-------------
West Hollywood
total number of places 8600
total iterations 19
error with city: West Hollywood and offset: 900
error with city: West Hollywood and offset: 900
error with city: West Hollywood and offset: 900
error with city: West Hollywood and offset: 900
error with city: West Hollywood and offset: 900
retries failed with  West Hollywood 900
-------------
Dodgertown
total number of pla

In [ ]:
def execute_sql(sql):
    con = pg.connect(
        database="airflow",
        user="postgres",
        password="admin",
        host="localhost",
        port= '5432'
        )

    cursor = con.cursor()
    cursor.execute(raw_data_create_SQL)
    con.commit()
    con.close()
    cursor.close()

In [ ]:
def load_data(sql, schema, table, df):
    con = pg.connect(
        database="airflow",
        user="postgres",
        password="admin",
        host="localhost",
        port= '5432'
        )

    cursor = con.cursor()
    cursor.execute(sql)
    output = io.StringIO()
    
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cursor.copy_expert(f"COPY {schema}.{table} FROM STDIN", output)
    
    con.commit()
    con.close()
    cursor.close()


In [ ]:
raw_data_create_SQL = '''
    CREATE TABLE IF NOT EXISTS public.yelp_raw_data(
        id text,
        alias text,
        name text,
        image_url text,
        is_closed text,
        url text,
        review_count BIGINT,
        categories text,
        rating decimal,
        coordinates text,
        transactions text,
        price varchar(10),
        location text,
        phone text,
        display_phone text,
        distance decimal,
        PRIMARY KEY (id)
    )

'''

In [ ]:
result_df = pd.DataFrame(all_res)
final_result_df = result_df.drop_duplicates(subset = ['id'])

In [ ]:
load_data(sql=raw_data_create_SQL, schema='public', table='yelp_raw_data', df=final_result_df)

In [ ]:
# create processed table
processed_df = pd.concat([final_result_df.drop(columns=['coordinates']).reset_index(), pd.DataFrame(final_result_df['coordinates'].values.tolist())], axis = 1).drop(columns=['index'])
processed_df = pd.concat([processed_df.reset_index(), pd.DataFrame(processed_df['location'].values.tolist())[['zip_code', 'city', 'state', 'country']]], axis = 1).drop(columns=['index'])
processed_df = pd.merge(left = processed_df, right = df_la[['zip', 'county']], left_on = 'zip_code', right_on = 'zip', how = 'left')
processed_df = processed_df.drop(columns = ['zip'])
processed_df

In [ ]:
clean_data_create_SQL = '''
    DROP TABLE IF EXISTS public.yelp_processed_data;
    CREATE TABLE public.yelp_processed_data(
        id text,
        alias text,
        name text,
        image_url text,
        is_closed text,
        url text,
        review_count BIGINT,
        categories text,
        rating decimal,
        transactions text,
        price varchar(10),
        location text,
        phone text,
        display_phone text,
        distance decimal,
        latitude decimal,
        longitude decimal,
        zip_code bigint,
        city text,
        state varchar(2),
        country varchar(5),
        county text,
        PRIMARY KEY (id)
    )

'''

In [ ]:
load_data(sql=clean_data_create_SQL, schema='public', table='yelp_processed_data', df=processed_df)

In [ ]:
# create category table
categories_df = pd.concat([categories_df[['id']].reset_index(), pd.DataFrame(categories_df['categories'].values.tolist())], axis = 1)
categories_df = categories_final_df.drop(columns = ['index'])
categories_df